## Informe de Extracción, Transformación y Carga de Datos (ETL): australian_user_reviews

En este apartado, llevaremos a cabo el proceso de Extracción, Transformación y Carga (ETL) del dataset australian_user_reviews donde se registran las reseñas de usuarios australianos sobre juegos en la plataforma Steam. El objetivo es preparar los datos para su posterior exploración y análisis.

Después de importar las bibliotecas necesarias para el trabajo, nos enfocaremos en abordar posibles inconvenientes en los datos, aplicaremos técnicas de depuración y preprocesamiento, y finalmente, almacenaremos los datos transformados en un archivo csv.

### 1. Importación de las bibliotecas necesarias

In [1]:
# Pandas para el análisis de datos tabulares
import pandas as pd

# NumPy proporciona soporte para arreglos y matrices multidimensionales
import numpy as np

# Langdetect es una biblioteca para detectar automáticamente el idioma en el que está escrito un texto
from langdetect import detect

# SentimentIntensityAnalyzer es una herramienta en NLTK para análisis de sentimientos
from nltk.sentiment import SentimentIntensityAnalyzer

### 2. Carga del archivo

In [2]:
# Ruta del archivo
ruta = "../Datasets/australian_user_reviews.json"

In [3]:
# Leemos el archivo
with open(ruta, 'r', encoding='utf-8') as file:
    data = file.readlines()

# Convertimos cada linea en un registro como código de Python (una lista de diccionarios)
registros = [eval(line.strip()) for line in data]

# Creamos un DataFrame a partir de los registros
df_user_reviews= pd.DataFrame(registros)

### 3. Exploración del DataFrame

In [4]:
# Mostramos las primeras 5 filas del DataFrame
print("\nPrimeras filas del DataFrame:")
df_user_reviews.head()


Primeras filas del DataFrame:


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


Según la visualización de las 5 primeras filas del DataFrame, este parece contener tres columnas:

- user_id: Esta columna parece contener un identificador único para cada usuario.

- user_url: Contiene enlaces a los perfiles de los usuarios en la plataforma Steam.

- reviews: Contiene información detallada sobre las revisiones realizadas por cada usuario. Cada entrada es un diccionario o estructura similar, que incluye detalles como la fecha de publicación, contenido de la revisión, etc.

In [5]:
# Mostramos el primer registro de la columna "reviews"
df_user_reviews.iloc[0,2]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

Respecto a esta última columna ("reviews"), se observa que contiene datos estructurados en formato JSON. Lo que haremos a continuación será normalizarlos en un nuevo DataFrame permitiéndonos explorar esta columna más a fondo, conservando las dos primeras columnas ("user_id" y "user_url").

In [6]:
# Creamos un nuevo DataFrame (df_UserReviews) detallando las columna "reviews" y manteniendo "user_id" y "user_url"
df_UserReviews = pd.json_normalize(registros, record_path=["reviews"], meta=["user_id", "user_url"])

# Especificamos el orden que tendrán las columnas
column_orden = ["user_id", "user_url", "funny", "posted", "last_edited", "item_id", "helpful", "recommend", "review"]

# Sobreescribimos el DataFrame con el nuevo orden de las columnas
df_UserReviews = df_UserReviews[column_orden]

# Mostramos las primeras 3 filas del nuevo DataFrame
df_UserReviews.head(3)


,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


In [7]:
# Revisamos las información general del DataFrame
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  bool  
 8   review       59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


Después de notar que las columnas principalmente contienen datos de tipo "object" y "bool", se deduce que aplicar estadísticas descriptivas a todas las columnas podría tener menos relevancia. Esto se debe a que las estadísticas descriptivas suelen ser más aplicables a columnas numéricas, mientras que las columnas 'object' suelen contener datos de texto que no son propicios para ciertos cálculos estadísticos.

### 4. Limpieza y preprocesamiento del conjunto de datos

Procedemos a verificar la ausencia simultanea de datos en columnas relevantes del DataFrame ("posted", "recommend" y "review"), ya que la falta de esos datos podría afectar negativamente las calidad de los resultados.

In [8]:
# Reemplazamos los valores vacíos, 'null' y 'None' con NaN en todo el DataFrame
df_UserReviews.replace(["", "null", "None"], np.nan, inplace=True)

In [9]:
# Mostramos las filas donde las columnas a considerar tienen valores nulos
columnas_a_considerar = ["posted", "recommend", "review"]
filas_con_nulos = df_UserReviews[df_UserReviews[columnas_a_considerar].isnull().all(axis=1)]
filas_con_nulos

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review


Los resultados indican que dentro del conjunto de datos actual, no hay filas donde "posted", "recommend" y "review" estén simultáneamente ausentes. Como resultado, podemos proceder con confianza en nuestro análisis.

In [10]:
# Establecemos un umbral del 80% para decidir que columnas eliminar por cantidad de valores nulos
umbral_nulos = 0.8

# Calculamos el porcentaje de valores nulos por columna
porcentaje_nulos = df_UserReviews.isnull().mean()

# Filtramos las columnas que superan el umbral
columnas_a_eliminar = porcentaje_nulos[porcentaje_nulos > umbral_nulos]

# Mostramos las columnas y su respectivo porcentaje de valores nulos
print(f"Las columnas con más del {umbral_nulos*100}% de valores nulos son:")
for columna, porcentaje in columnas_a_eliminar.items():
    print(f"{columna}: {round(porcentaje*100, 2)}%")

Las columnas con más del 80.0% de valores nulos son:
funny: 86.26%
last_edited: 89.65%


Se observa que la mayoría de filas en las columnas "funny" y "last_edited" contienen valores nulos y no son cruciales para el análisis, por lo que se procede a eliminarlas.

In [11]:
# Eliminamos las columnas "funny" y "last_edited"
df_UserReviews.drop(columns=columnas_a_eliminar.index, inplace=True)

Adicionalmente, las columnas "helpful" y "user_url" tampoco aportan datos relevantes para nuestro estudio objetivo, por lo que también se procede a eliminarlas.

In [12]:
# Eliminamos las columnas "helpful" y "user_url"
df_UserReviews.drop(["helpful","user_url"], axis=1, inplace=True)

Ahora, verificaremos si existen registros duplicados y trabajaremos con ellos.

In [13]:
# Ordenamos el DataFrame y buscamos registros duplicados
df_UserReviews.sort_values("user_id")
filas_duplicadas = df_UserReviews[df_UserReviews.duplicated(subset=["user_id", "item_id", "posted", "review"], keep=False)]
filas_duplicadas.count()

user_id      1736
posted       1736
item_id      1736
recommend    1736
review       1736
dtype: int64

In [14]:
# Eliminamos los registros duplicados basándonos en mútiples columnas
duplicados_eliminados = df_UserReviews.drop_duplicates(subset=["user_id", "item_id", "posted", "review"], keep=False, inplace=True)

print(f"Cantidad total de registros duplicados eliminados: {filas_duplicadas.shape[0]}")

Cantidad total de registros duplicados eliminados: 1736


Dentro del DataFrame, algo muy importante a considerar es el año en que se ha hecho el "review" de cada usuario ya que esto facilita los análisis temporales y tendencias a lo largo del tiempo.

Aunque no contamos con una columna de año, la podremos obtener a partir de la columna "posted", siendo luego esta última eliminada para simplificar la estructura del conjunto de datos.

In [15]:
# Convertimos la columna "posted" a tipo str para poder eliminar caracteres no deseados y luego la convertimos a tipo datatime
df_UserReviews["posted"] = pd.to_datetime(df_UserReviews['posted'].astype(str).str.replace(r'Posted |,|\.', '', regex=True), errors="coerce")

# Creamos la columna "year" de tipo entero partir de la columna "posted"
df_UserReviews["year"] = df_UserReviews["posted"].dt.year.astype('Int64')

# Observamos el resultado
df_UserReviews.head(5)

,user_id,posted,item_id,recommend,review,year
0,76561197970982479,2011-11-05,1250,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,2011-07-15,22200,True,It's unique and worth a playthrough.,2011
2,76561197970982479,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,2014-06-24,251610,True,I know what you think when you see this title ...,2014
4,js41637,2013-09-08,227300,True,For a simple (it's actually not all that simpl...,2013


In [16]:
# Ordenamos el DataFrame por "item_id" y "year" para asegurar que la interpolación se haga correctamente
df_UserReviews = df_UserReviews.sort_values(["item_id", "year"])

# Rellenamos valores nulos en "year" mediante interpolación lineal por grupo (item_id)
df_UserReviews["year"] = df_UserReviews.groupby("item_id", group_keys=False)["year"].apply(lambda group: group.interpolate(method="pad") if group.notna().any() else group)

# Si aún hay valores nulos después de la interpolación, se llenan con la mediana. 
df_UserReviews["year"] = df_UserReviews["year"].fillna(df_UserReviews["year"].median())

In [18]:
# Eliminamos las columnas 'posted' y 'user_id' que ya no son de utilidad
df_UserReviews.drop(['posted', 'user_id'], axis=1, inplace=True)

#Observamos el resultado
df_UserReviews.head()

,item_id,recommend,review,year
5331,10,True,this game is the 1# online action game is awes...,2011
22702,10,True,GYERTEK GAMELNI MINDENKI ITT VAN AKI SZÁMIT !!...,2012
35539,10,True,:D,2012
43134,10,True,Good Game :D,2012
24137,10,True,jueguenlooooooo,2013


Ahora, evaluaremos la columna "review" para obtener estadísticas sobre la cantidad de reseñas por lenguaje.

Este análisis será crucial para determinar si el proceso de análisis de sentimientos se realizará en todo el conjunto de datos o si se aplicará un filtro específico por idioma.

In [19]:
# Creamos una función para el análisis de texto
def detectar_idioma(texto):
    try:
        return detect(texto)
    except:
        return None
    
# Aplicamos la función para detectar idioma y crear una nueva columna: 'language'
df_UserReviews["language"] = df_UserReviews["review"].apply(detectar_idioma)

# Calculamos el conteo y porcentaje de cada idioma
conteo_por_idioma = df_UserReviews["language"].value_counts()
porcentaje_por_idioma = df_UserReviews["language"].value_counts(normalize=True) * 100

# Creamos un nuevo DataFrame con el conteo y porcentaje
resumen_idiomas = pd.DataFrame({"Conteo": conteo_por_idioma,
                                "Porcentaje": porcentaje_por_idioma.round(2).astype(str) + "%"})

# Ordenamos el DataFrame por el conteo de mayor a menor
resumen_idiomas = resumen_idiomas.sort_values(by="Conteo", ascending=False)

#Observamos el resultado
resumen_idiomas.head()

,Conteo,Porcentaje
language,,
en,44417,77.91%
pt,2115,3.71%
es,1232,2.16%
de,1128,1.98%
so,987,1.73%


Los nombres completos de los idiomas obtenidos son:

- "en": English
- "pt": Portugués
- "es": Español
- "de": German
- "so": Somali

Con los resultados obtenidos se consideró realizar el análisis de sentimiento solo en los registros en inglés ("en") por las siguientes razones:

- Mayor Representatividad.
- Precisión del Modelo.
- Eficiencia Computacional.

In [20]:
# Filtramos los registros donde "language" no es igual a 'en'. Y sobreescrimos el DataFrame
df_UserReviews = df_UserReviews[df_UserReviews["language"] == "en"]

In [21]:
# Eliminamos la columna "language" ya que no la necesitaremos más
df_UserReviews =  df_UserReviews.drop("language", axis=1)

### Feature Engineering

La columna "review" incluye reseñas de juegos hechos por distintos usuarios, por lo que partiendo de ahí se va crear la columna "sentiment_analysis" aplicando análisis de sentimiento con NLP (Procesamiento del Lenguaje Natural) con la siguiente escala:

Debe tomar el valor '0' si es malo,
Debe tomar el valor '1' si es neutral y
Debe tomar el valor '2' si es positivo.

De no ser posible este análisis por estar ausente la reseña escrita, tomará el valor de 1. Esta nueva columna reemplaza la de "review" para facilitar el trabajo de los modelos de machine learning y el análisis de datos.

In [22]:
# Convertimos todas las letras a minúsculas
df_UserReviews["review"] = df_UserReviews["review"].str.lower()

In [23]:
# Eliminamos caracteres especiales
df_UserReviews["review"] = df_UserReviews["review"].replace("[^A-Za-z0-9\s]+", "", regex=True)

In [24]:
# Eliminamos caracteres de puntuación irrelevantes para el análisis de sentimento
df_UserReviews["review"] = df_UserReviews['review'].str.replace('[^\w\s]', '', regex=True)

In [25]:
# Creamos una función para el análisis de sentimientos
def analyze_sentiments(df):

    # Instanciamos el analizador de sentimientos
    sia = SentimentIntensityAnalyzer()

    # Aplicamos el análisis de sentimientos y asignamos valores numéricos
    df["compound_score"] = df["review"].apply(lambda review: sia.polarity_scores(review)["compound"])
    df["sentiment_analysis"] = df["compound_score"].apply(lambda score: 0 if score < 0 else (1 if score == 0 else 2))

    # Conteo de reviews por score
    score_counts = df["sentiment_analysis"].value_counts()

    # Conteo de reviews en blanco
    blank_reviews_count = df["review"].isnull().sum()

    # Total de reviews
    total_reviews = len(df)

    # Calcular porcentajes
    score_percentages = (score_counts / total_reviews * 100).round(2)
    blank_reviews_percentage = (blank_reviews_count / total_reviews * 100).round(2)
     
    # Eliminamos las columnas "review" y "compound_score", no necesitaremos estos datos
    df.drop(["review", "compound_score"], axis=1, inplace=True)  

    return df, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage

In [26]:
# Llamamos a la función analyze_sentiments
df_UserReviews, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage = analyze_sentiments(df_UserReviews)

In [27]:
# Creamos un nuevo Dataframe con el conteo y porcentaje
resumen_sentimientos = pd.DataFrame({"Conteo": score_counts,
                                     "Porcentaje": score_percentages.round(2).astype(str) + '%'})

# Ordenamos el DataFrame por el conteo de mayor a menor
resumen_sentimientos = resumen_sentimientos.sort_values(by='Conteo', ascending=False)

# Imprimimos los resultados
print("Resumen de análisis de sentimientos:\n")
print(resumen_sentimientos)
print(f"\nConteo de reviews en blanco: {blank_reviews_count}. Porcentaje: {blank_reviews_percentage.round(2).astype(str)}%")

Resumen de análisis de sentimientos:

                    Conteo Porcentaje
sentiment_analysis                   
2                    31481     70.88%
0                     8269     18.62%
1                     4667     10.51%

Conteo de reviews en blanco: 0. Porcentaje: 0.0%


En el análisis de sentimientos de las reseñas en la columna "sentiment_analysis", observamos lo siguiente:

- Positivo (2):
Representa la categoría dominante en las reseñas, abarcando el 70.83% del total. Esto indica que la gran mayoría de las reseñas tiene un sentimiento positivo.

- Neutral (1):
Aunque en menor proporción, un 10.53% de las reseñas se clasifican como neutrales. Esto podría indicar que algunos usuarios expresan opiniones sin un sesgo claramente positivo o negativo.

- Negativo (0):
La categoría de sentimiento negativo tiene una presencia más baja, constituyendo el 18.63%. Esto sugiere que, en general, la plataforma tiene una inclinación positiva en las reseñas de los usuarios.

Consideraciones: La mayoría de los usuarios tiende a expresar opiniones positivas en sus reseñas.La proporción de reseñas neutrales es relativamente baja, lo que indica que la mayoría de los usuarios tiende a tener opiniones claramente positivas o negativas.

In [28]:
# Observamos el resultado
df_UserReviews.head()

,item_id,recommend,year,sentiment_analysis
5331,10,True,2011,2
45506,10,True,2013,1
7801,10,True,2014,2
7967,10,True,2014,2
8519,10,True,2014,2


### 5. Guardar el conjunto de datos limpio

In [29]:
# Los archivos se almacenan en local 
df_UserReviews.to_csv('australian_user_reviews_cleaned.csv', index=False)